**Import required libraries and scripts**

In [ ]:
#Import required libraries and scripts
from scripts.library_preparation import *
from scripts.utilities import *
from scripts.docking_functions import *
from scripts.clustering_functions import *
from scripts.rescoring_functions import *
from scripts.consensus_methods import *
from scripts.performance_calculation import *
from scripts.dogsitescorer import *
from scripts.get_pocket import *

software = '/home/alacournola/DockM8/software'
protein_file = '/home/alacournola/DEKOIS-noprot2/aldr/receptor_protoss_prepared.pdb'
ref_file = '/home/alacournola/DEKOIS-noprot2/aldr/crystal_ligand_protoss.sdf'
docking_library = '/home/alacournola/DEKOIS-noprot2/aldr/merged_actives_decoys.sdf'
docking_programs = ['GNINA', 'SMINA', 'PLANTS']
clustering_metrics = ['RMSD', 'spyRMSD', 'espsim', '3DScore', 'bestpose', 'bestpose_GNINA', 'bestpose_SMINA', 'bestpose_PLANTS']
rescoring_functions = ['gnina', 'AD4', 'chemplp', 'rfscorevs', 'LinF9', 'RTMScore', 'SCORCH', 'vinardo', 'plecscore', 'nnscore', 'KORPL', 'ConvexPL']
id_column = 'ID'
n_poses = 10
exhaustiveness = 8
protonation = 'pkasolver'
# ncpus = int(os.cpu_count()-2)
ncpus = 14
pocket = 'reference'
#Create a temporary folder for all further calculations
w_dir = os.path.dirname(protein_file)
print('The working directory has been set to:', w_dir)
create_temp_folder(w_dir+'/temp')
if pocket == 'reference':
    pocket_definition = get_pocket(ref_file, protein_file, 8)
    print(pocket_definition)
elif pocket == 'dogsitescorer':
    pocket_definition = binding_site_coordinates_dogsitescorer(protein_file, w_dir, method='volume')

In [ ]:
# if os.path.isfile(protein_file.replace('.pdb', '_pocket.pdb')) == False:


In [ ]:
if os.path.isfile(w_dir+'/temp/final_library.sdf') == False:
    prepare_library(docking_library, id_column, software, protonation, ncpus)

In [ ]:
docking(w_dir, protein_file, ref_file, software, docking_programs, exhaustiveness, n_poses, ncpus, pocket_definition)


In [ ]:
print('Loading all poses SDF file...')
tic = time.perf_counter()
all_poses = PandasTools.LoadSDF(w_dir+'/temp/allposes.sdf', idName='Pose ID', molColName='Molecule', includeFingerprints=False, strictParsing=True)
toc = time.perf_counter()
print(f'Finished loading all poses SDF in {toc-tic:0.4f}!...')

In [ ]:
for metric in clustering_metrics:
        if os.path.isfile(w_dir+f'/temp/clustering/{metric}_clustered.sdf') == False:
            cluster_pebble(metric, 'KMedoids', w_dir, protein_file, all_poses, ncpus)

In [ ]:
for metric in clustering_metrics:
        rescore_all(w_dir, protein_file, pocket_definition, software, w_dir+f'/temp/clustering/{metric}_clustered.sdf', rescoring_functions, ncpus)


In [ ]:
calculate_EF_single_functions(w_dir, docking_library, clustering_metrics)

In [ ]:
apply_consensus_methods_combinations(w_dir, docking_library, clustering_metrics)

In [ ]:
apply_consensus_methods(w_dir, 'bestpose', 'method6', ['gnina', 'RTMScore'])